# Twinning Theory （缠论）

In [1]:
# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


%matplotlib inline
%run tick2bar.ipynb


from typing import List, Dict, Any
import datetime as dt

import pandas as pd
import mplfinance as mpf


mpf.__version__

'0.12.7a17'

In [2]:
# mplfinance settings.
mpf_color = mpf.make_marketcolors(
    up='red',  # 上涨K线的颜色
    down='green',  # 下跌K线的颜色
    inherit=True
)

mpf_style = mpf.make_mpf_style(
    marketcolors=mpf_color,
    rc={
        'font.family': 'SimHei',       # 指定默认字体：解决plot不能显示中文问题
        'axes.unicode_minus': False,  # 解决保存图像是负号'-'显示为方块的问题
    }
)

In [3]:
# 画图数据
df_plot = df_1min.loc['2021-09-25 01:00:00.000000':, ['open', 'high', 'low', 'close']]

# 调整时间戳到北京时间
df_plot.index = df_plot.index - dt.timedelta(hours=4)

print(df_plot)

                        open     high      low    close
datetime                                               
2021-09-24 21:00:00  23140.0  23140.0  23045.0  23100.0
2021-09-24 21:01:00  23090.0  23100.0  23050.0  23050.0
2021-09-24 21:02:00  23050.0  23115.0  23040.0  23105.0
2021-09-24 21:03:00  23105.0  23110.0  23060.0  23060.0
2021-09-24 21:04:00  23060.0  23065.0  23030.0  23035.0
...                      ...      ...      ...      ...
2021-09-25 00:55:00  22955.0  22960.0  22945.0  22950.0
2021-09-25 00:56:00  22950.0  22960.0  22945.0  22950.0
2021-09-25 00:57:00  22950.0  22955.0  22950.0  22950.0
2021-09-25 00:58:00  22955.0  22960.0  22945.0  22955.0
2021-09-25 00:59:00  22950.0  22960.0  22945.0  22945.0

[240 rows x 4 columns]


In [4]:
# 没什么好办法遍历 DataFrame 的同时顺便合并。
# 转 Python dict 试试看。

In [6]:
class Bar:
    idx: pd.Timestamp
    open: float
    high: float
    low: float
    close: float
    
    def __init__(self, idx: pd.Timestamp, open: float, high: float, low: float, close: float):
        self.idx = idx
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        
    def __str__(self) -> str:
        return f'Candlestick at {self.idx}: O {self.open}, H {self.high}, L {self.low}, C {self.close}'


# 获取 DataFrame 的 index list。
df_index_list: List[pd.Timestamp] = df_plot.index.tolist()
# [print(idx.timestamp) for idx in df_plot.index]

# 生成数据。
candlstick_list: List[Bar] = [
    Bar(
        idx=idx,
        open=df_plot.loc[idx, 'open'],
        high=df_plot.loc[idx, 'high'],
        low=df_plot.loc[idx, 'low'],
        close=df_plot.loc[idx, 'close']        
    ) for idx in df_plot.index
]

for item in candlstick_list:
    print(item)

Candlestick at 2021-09-24 21:00:00: O 23140.0, H 23140.0, L 23045.0, C 23100.0
Candlestick at 2021-09-24 21:01:00: O 23090.0, H 23100.0, L 23050.0, C 23050.0
Candlestick at 2021-09-24 21:02:00: O 23050.0, H 23115.0, L 23040.0, C 23105.0
Candlestick at 2021-09-24 21:03:00: O 23105.0, H 23110.0, L 23060.0, C 23060.0
Candlestick at 2021-09-24 21:04:00: O 23060.0, H 23065.0, L 23030.0, C 23035.0
Candlestick at 2021-09-24 21:05:00: O 23030.0, H 23045.0, L 23010.0, C 23035.0
Candlestick at 2021-09-24 21:06:00: O 23040.0, H 23040.0, L 23010.0, C 23030.0
Candlestick at 2021-09-24 21:07:00: O 23030.0, H 23040.0, L 22970.0, C 22995.0
Candlestick at 2021-09-24 21:08:00: O 22995.0, H 23000.0, L 22955.0, C 22965.0
Candlestick at 2021-09-24 21:09:00: O 22960.0, H 22990.0, L 22945.0, C 22975.0
Candlestick at 2021-09-24 21:10:00: O 22975.0, H 23005.0, L 22970.0, C 22985.0
Candlestick at 2021-09-24 21:11:00: O 22985.0, H 23020.0, L 22980.0, C 23015.0
Candlestick at 2021-09-24 21:12:00: O 23015.0, H 230

In [51]:
def is_inclusive(current_bar: Bar, previous_bar: Bar) -> bool:
    """
    判断是否K线包含。
    """
    return (current_bar.high <= previous_bar.high and current_bar.low >= previous_bar.low) or (current_bar.high >= previous_bar.high and current_bar.low <= previous_bar.low)


from enum import Enum
class Trend(Enum):
    Up = 'Bullish'
    Down = 'Bearish'
    Bullish = 'Bullish'
    Bearish = 'Bearish'
    
    def __str__(self) -> str:
        return '上涨' if self.value == 'Bullish' else '下跌'

    
def get_trend(current_bar: Bar, previous_bar: Bar) -> Trend:
    """
    判断趋势。
    """
    if current_bar.high >= previous_bar.high and current_bar.low >= previous_bar.low:
        return Trend.Bullish
    elif current_bar.high <= previous_bar.high and current_bar.low <= previous_bar.low:
        return Trend.Bearish
    else:
        # 针对开始的几根K线，存在包含关系。
        delta_h: int = current_bar.high - previous_bar.high
        delta_l: int = current_bar.low - previous_bar.low
        if abs(delta_h) > abs(delta_l):
            # High 价差大，delta_h 为正数则为上涨，反之下跌。
            return Trend.Bullish if delta_h > 0 else Trend.Bearish
        elif abs(delta_h) < abs(delta_l):
            # High 价差小，delta_h 为正数则为下跌，反之上涨。
            return Trend.Bearish if delta_h > 0 else Trend.Bullish
        else:
            # 两根K线一样高低，强制定为上涨。
            return Trend.Bullish


class BarMerged:
    idx: pd.Timestamp
    high: float
    low: float
    interval: int
    
    def __init__(self, idx: pd.Timestamp, high: float, low: float, interval: int):
        self.idx = idx
        self.high = high
        self.low = low
        self.interval = interval
        
    def __str__(self) -> str:
        return f'BarMerged at {self.idx}, Keep {self.interval} period, H {self.high}, L {self.low}'


first_bar = candlstick_list[0]
merged_list: List[BarMerged] = []
data_length: int = len(candlstick_list)
trend: Trend
# 处理第0，1两根
print(candlstick_list[0].idx, 'INIT 0')
merged_list.append(
    BarMerged(
        idx=candlstick_list[0].idx,
        high=candlstick_list[0].high,
        low=candlstick_list[0].low,
        interval=1
    )
)
print(candlstick_list[1].idx, 'INIT 1')
merged_list.append(
    BarMerged(
        idx=candlstick_list[1].idx,
        high=candlstick_list[1].high,
        low=candlstick_list[1].low,
        interval=1
    )
)

print(f'前K：count={len(merged_list)}, idx={merged_list[-1].idx}, H={merged_list[-1].high}, L={merged_list[-1].low}')

# 处理第3根（编号2）开始
for i in range(2, data_length):
    current_bar = candlstick_list[i]
    previous_bar = merged_list[-1]
    print(current_bar.idx)
    print(f'\t当前： H={current_bar.high}, L={current_bar.low} vs 前K：H={previous_bar.high}, L={previous_bar.low}')
    if is_inclusive(current_bar=current_bar, previous_bar=merged_list[-1]):
        print(f'\t包含： YES')
        trend = get_trend(current_bar=merged_list[-1], previous_bar=merged_list[-2])
        print('\t趋势：', trend)
        if trend == Trend.Bullish:
            merged_list[-1].high = max(current_bar.high, previous_bar.high)
            merged_list[-1].low = max(current_bar.low, previous_bar.low)
            merged_list[-1].interval += 1
            print(f'\t结果： 开始于{merged_list[-1].idx}, 持续{merged_list[-1].interval}周期, H={merged_list[-1].high}, L={merged_list[-1].low}')
        if trend == Trend.Bearish:
            merged_list[-1].high = min(current_bar.high, previous_bar.high)
            merged_list[-1].low = min(current_bar.low, previous_bar.low)
            merged_list[-1].interval += 1
            print(f'\t结果： 开始于{merged_list[-1].idx}, 持续{merged_list[-1].interval}周期, H={merged_list[-1].high}, L={merged_list[-1].low}')
        if trend is None:
            print(f'\t结果： ERROR ------------------')
    else:
        print(f'\t包含： ---')
        merged_list.append(
            BarMerged(
                idx=current_bar.idx,
                high=current_bar.high,
                low=current_bar.low,
                interval=1
            )
        )

2021-09-24 21:00:00 INIT 0
2021-09-24 21:01:00 INIT 1
前K：count=2, idx=2021-09-24 21:01:00, H=23100.0, L=23050.0
2021-09-24 21:02:00
	当前： H=23115.0, L=23040.0 vs 前K：H=23100.0, L=23050.0
	包含： YES
	趋势： 下跌
	结果： 开始于2021-09-24 21:01:00, 持续2周期, H=23100.0, L=23040.0
2021-09-24 21:03:00
	当前： H=23110.0, L=23060.0 vs 前K：H=23100.0, L=23040.0
	包含： ---
2021-09-24 21:04:00
	当前： H=23065.0, L=23030.0 vs 前K：H=23110.0, L=23060.0
	包含： ---
2021-09-24 21:05:00
	当前： H=23045.0, L=23010.0 vs 前K：H=23065.0, L=23030.0
	包含： ---
2021-09-24 21:06:00
	当前： H=23040.0, L=23010.0 vs 前K：H=23045.0, L=23010.0
	包含： YES
	趋势： 下跌
	结果： 开始于2021-09-24 21:05:00, 持续2周期, H=23040.0, L=23010.0
2021-09-24 21:07:00
	当前： H=23040.0, L=22970.0 vs 前K：H=23040.0, L=23010.0
	包含： YES
	趋势： 下跌
	结果： 开始于2021-09-24 21:05:00, 持续3周期, H=23040.0, L=22970.0
2021-09-24 21:08:00
	当前： H=23000.0, L=22955.0 vs 前K：H=23040.0, L=22970.0
	包含： ---
2021-09-24 21:09:00
	当前： H=22990.0, L=22945.0 vs 前K：H=23000.0, L=22955.0
	包含： ---
2021-09-24 21:10:00
	当前： H=23005.0, L

In [56]:
# 合并前后对照。
mm: int = 0
for i in range(data_length):
    timestamp = candlstick_list[i].idx
    print(f'{i:3d}, {candlstick_list[i].idx},\t原始K线: H={candlstick_list[i].high:7.1f}, L={candlstick_list[i].low:7.1f},\t', end='')
    if merged_list[mm].idx == timestamp:
        print(f'\t合并K线: H={merged_list[mm].high:7.1f}, L={merged_list[mm].low:7.1f}, 持续{merged_list[mm].interval:2d} 周期')
        if mm < len(merged_list) - 1:
            mm += 1
    else:
        print()

  0, 2021-09-24 21:00:00,	原始K线: H=23140.0, L=23045.0,		合并K线: H=23140.0, L=23045.0, 持续 1 周期
  1, 2021-09-24 21:01:00,	原始K线: H=23100.0, L=23050.0,		合并K线: H=23100.0, L=23040.0, 持续 2 周期
  2, 2021-09-24 21:02:00,	原始K线: H=23115.0, L=23040.0,	
  3, 2021-09-24 21:03:00,	原始K线: H=23110.0, L=23060.0,		合并K线: H=23110.0, L=23060.0, 持续 1 周期
  4, 2021-09-24 21:04:00,	原始K线: H=23065.0, L=23030.0,		合并K线: H=23065.0, L=23030.0, 持续 1 周期
  5, 2021-09-24 21:05:00,	原始K线: H=23045.0, L=23010.0,		合并K线: H=23040.0, L=22970.0, 持续 3 周期
  6, 2021-09-24 21:06:00,	原始K线: H=23040.0, L=23010.0,	
  7, 2021-09-24 21:07:00,	原始K线: H=23040.0, L=22970.0,	
  8, 2021-09-24 21:08:00,	原始K线: H=23000.0, L=22955.0,		合并K线: H=23000.0, L=22955.0, 持续 1 周期
  9, 2021-09-24 21:09:00,	原始K线: H=22990.0, L=22945.0,		合并K线: H=22990.0, L=22945.0, 持续 1 周期
 10, 2021-09-24 21:10:00,	原始K线: H=23005.0, L=22970.0,		合并K线: H=23005.0, L=22970.0, 持续 1 周期
 11, 2021-09-24 21:11:00,	原始K线: H=23020.0, L=22980.0,		合并K线: H=23020.0, L=22980.0, 持续 2 周期
 12, 2021-09-24 

In [ ]:
class FractalType(Enum):
    Upper = 'Upper'
    Lower = 'Lower'
    Not = 'Not'
    Unknown = 'Unknow'
    
    def __str__(self) -> str:
        if self.value == 'Upper':
            return '顶分型'
        elif self.value == 'Lower':
            return '底分型'
        elif self.value == 'Not':
            return '非分型'
        elel:
            return '未知状态'

    
class FractalPosition(Enum):
    Reversal = 'Reversal'
    Continuous = 'Continuous'
    
    def __str__(self) -> str:
        return '反转' if self.value == 'Reversal' else '中继'


def get_fractal():
    pass